<a href="https://colab.research.google.com/github/tylaar1/PICAR-autopilot/blob/main/BA_youtube_tutorial_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pls
import os

# Load in dataset

In [29]:
labels_file_path = '/content/drive/MyDrive/0. MSc MLiS/google SPRING SEMESTER/1. PHYS4036 MLiS2/MLiS2 Project/KAGGLEDATAmachine-learning-in-science-ii-2025/training_norm.csv'
labels_df = pd.read_csv(labels_file_path)

image_folder_path = '/content/drive/MyDrive/0. MSc MLiS/google SPRING SEMESTER/1. PHYS4036 MLiS2/MLiS2 Project/KAGGLEDATAmachine-learning-in-science-ii-2025/training_data/training_data'
image_file_paths = [os.path.join(image_folder_path, f) for f in os.listdir(image_folder_path)]

missing rows in labels_df: 3141, 3999, 4895, 8285, 10171

In [30]:
imagepaths_df = pd.DataFrame(image_file_paths, columns=['image_file_paths'])

indices_to_remove = [3140, 3997, 4892, 8281, 10166]
imagepaths_df_cleaned = imagepaths_df.drop(indices_to_remove, axis=0) # drops missing rows
imagepaths_df_cleaned = imagepaths_df_cleaned.reset_index(drop=True) # resets the row index
imagepaths_df_cleaned = imagepaths_df_cleaned[:13793] # gets rid of the last 5 images of the dataset which dont have labels

imagepaths_df_cleaned.head(5)

,image_file_paths
0,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
1,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
2,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
3,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
4,/content/drive/MyDrive/0. MSc MLiS/google SPRI...


In [33]:
concat_df = pd.concat([labels_df, imagepaths_df_cleaned], axis=1)
concat_df

,image_id,angle,speed,image_file_paths
0,1,0.4375,0.0,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
1,2,0.8125,1.0,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
2,3,0.4375,1.0,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
3,4,0.6250,1.0,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
4,5,0.5000,0.0,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
...,...,...,...,...
13788,13794,0.6250,1.0,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
13789,13795,0.4375,1.0,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
13790,13796,0.5625,0.0,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
13791,13797,0.6250,0.0,/content/drive/MyDrive/0. MSc MLiS/google SPRI...


# EDA

In [34]:
concat_df.value_counts('speed')

,count
speed,
1.000000,10402
0.000000,3390
1.428571,1


note: imbalance datset

# resizing images

In [54]:
# using df with just 100 elements (instead of 13.8k) so computation is quicker while im trying to figure out what works

trial_df = concat_df[:100]
trial_df.head(5)

,image_id,angle,speed,image_file_paths
0,1,0.4375,0.0,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
1,2,0.8125,1.0,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
2,3,0.4375,1.0,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
3,4,0.6250,1.0,/content/drive/MyDrive/0. MSc MLiS/google SPRI...
4,5,0.5000,0.0,/content/drive/MyDrive/0. MSc MLiS/google SPRI...


In [50]:
import cv2

im_size = 220

images = []

for path in trial_df['image_file_paths']:
  img = cv2.imread(path)
  img = cv2.resize(img, (im_size, im_size)) / 255.0 # div by 255 so numbers are all [0,1]
  images.append(img)

In [52]:
images[0].shape

(220, 220, 3)

# train, test split

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

images = trial_df['image_file_paths'].values
y = trial_df[['angle', 'speed']].values  # Both angle and speed as labels

# Shuffle the dataset (images and labels are shuffled together)
images, y = shuffle(images, y, random_state=1)

# Split the dataset into training and test sets
train_x, test_x, train_y, test_y = train_test_split(images, y, test_size=0.2, random_state=1)

# Check the shapes of the split data
print(f"Training data (X) shape: {train_x.shape}")
print(f"Testing data (X) shape: {test_x.shape}")
print(f"Training labels (y) shape: {train_y.shape}")
print(f"Testing labels (y) shape: {test_y.shape}")

Training data (X) shape: (80,)
Testing data (X) shape: (20,)
Training labels (y) shape: (80, 2)
Testing labels (y) shape: (20, 2)


In [63]:
train_y[0]

array([0.875, 0.   ])

In [60]:
import tensorflow as tf

In [64]:
num_channels = 3 # RGB
img_size = 220
num_outputs = 2 # angle and speed

# Model input
x = tf.keras.Input(shape=(img_size, img_size, num_channels), dtype=tf.float32)
y = tf.keras.Input(shape=(num_outputs,), dtype=tf.float32)